In [1]:
!pip install tensorflow_datasets==4.0.1
!pip freeze | grep tensorflow

     |████████████████████████████████| 3.5 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 47 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 198 kB 12.7 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=a968d78b9ce23cefa218a2d6a55b915f02614ad1a94de498344e4943c4289e47
  Stored in directory: /home/jovyan/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=f6b2403bfef94cac03845cb67d06d495561202d7ba6f8cf4652423456abba826
  Stored in directory: /home/jovyan/.cache/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built future promise
tensorflow==2.4.1
tensorflow-datasets==4.0.1
tensorflow-estimator==2.4.0
tensorflow-metadata==

In [3]:
import tensorflow_datasets as tfds

test, train = tfds.load('ag_news_subset', split=['test', 'train'], shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/ag_news_subset/1.0.0.incomplete4TOS6P/ag_news_subset-train.tfrecord


  0%|          | 0/120000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/ag_news_subset/1.0.0.incomplete4TOS6P/ag_news_subset-test.tfrecord


  0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news_subset downloaded and prepared to /home/jovyan/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [6]:
df_train = tfds.as_dataframe(train.take(-1))
df_test = tfds.as_dataframe(test.take(-1))

In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
train_titles = []
train_labels = []
for title in df_train['title']:
    train_titles.append(str(title))
    
for label in tf.constant(df_train['label']):
    train_labels.append(label.numpy())

train_labels_final = np.asarray(train_labels)

In [8]:
test_titles = []
test_labels = []
for title in df_test['title']:
    test_titles.append(str(title))

for label in tf.constant(df_test['label']):
    test_labels.append(label.numpy())
    
test_labels_final = np.array(test_labels)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer

vocab_size=10000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_titles)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_length = 100

train_sequences = tokenizer.texts_to_sequences(train_titles)
padded_train_sequences = pad_sequences(train_sequences, max_sequence_length, truncating='post')

test_sequences = tokenizer.texts_to_sequences(test_titles)
padded_test_sequences = pad_sequences(test_sequences, max_sequence_length, truncating='post')

In [12]:
# Create the ANN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers


embedding_dimension = 16

classifier = Sequential()

classifier.add(tf.keras.layers.Embedding(vocab_size, embedding_dimension,
                          input_length=max_sequence_length))

classifier.add(layers.GlobalAveragePooling1D())

classifier.add(Dense(units = 64, activation='relu', 
                     kernel_initializer = 'uniform'))


classifier.add(Dense(units = 32, activation='relu', 
                     kernel_initializer='uniform'))
classifier.add(Dropout(rate = .3))

classifier.add(Dense(units =32, activation='relu', 
                     kernel_initializer='uniform'))
classifier.add(Dropout(rate = .3))

classifier.add(Dense(units =32, activation='relu', 
                     kernel_initializer='uniform'))
classifier.add(Dropout(rate = .2))

classifier.add(Dense(units =32, activation='relu', 
                     kernel_initializer='uniform'))
classifier.add(Dropout(rate = .2))


# Output Layer. Four output nodes for our four classification types of news headlines.
classifier.add(Dense(units = 4, activation='softmax', 
                     kernel_initializer='uniform'))

classifier.compile(optimizer = 'rmsprop', loss ='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
from tensorflow.keras import callbacks

terminateOnNanCallback = callbacks.TerminateOnNaN()
earlyStopCallback = callbacks.EarlyStopping(monitor='val_acc', patience=3)

In [ ]:
# Fit the ANN to the training data
import tensorflow as tf
import numpy as np
history = classifier.fit(padded_train_sequences,
                         train_labels_final,
                         epochs=100,
                         batch_size=64,
                         validation_data = (padded_test_sequences, test_labels_final))

Epoch 1/100
1875/1875 [==============================] - 5s 2ms/step - loss: 1.3864 - accuracy: 0.2479 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 2/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2489 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 3/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3863 - accuracy: 0.2519 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 4/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3863 - accuracy: 0.2523 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 5/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2483 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 6/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2457 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 7/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3863 - accuracy: 0.2509 - val_loss: 1.3863 - val_ac

1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2489 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 58/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2492 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 59/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2485 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 60/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2493 - val_loss: 1.3864 - val_accuracy: 0.2500
Epoch 61/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3863 - accuracy: 0.2505 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 62/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3863 - accuracy: 0.2507 - val_loss: 1.3863 - val_accuracy: 0.2500
Epoch 63/100
1875/1875 [==============================] - 4s 2ms/step - loss: 1.3864 - accuracy: 0.2504 - val_loss: 1.3863 - val_accuracy

In [173]:
# Validate the ANN
scores = classifier.evaluate(padded_test_sequences, test_labels_final)
print("Model accuracy on test data = {} ".format(scores[1]))

238/238 [==============================] - 0s 910us/step - loss: 1.3652 - accuracy: 0.3186
Model accuracy on test data = 0.3185526430606842 


In [175]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_keras_api_names',
 '_keras_api_names_v1',
 'char_level',
 'document_count',
 'filters',
 'fit_on_sequences',
 'fit_on_texts',
 'get_config',
 'index_docs',
 'index_word',
 'lower',
 'num_words',
 'oov_token',
 'sequences_to_matrix',
 'sequences_to_texts',
 'sequences_to_texts_generator',
 'split',
 'texts_to_matrix',
 'texts_to_sequences',
 'texts_to_sequences_generator',
 'to_json',
 'word_counts',
 'word_docs',
 'word_index']

In [ ]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import pandas as pd

# URLS of known topics
business_news_url="https://news.google.com/news/rss/headlines/section/topic/BUSINESS?ned=us&hl=en&gl=US"
tech_news_url = "https://news.google.com/news/rss/headlines/section/topic/TECHNOLOGY?ned=us&hl=en&gl=US"
science_news_url = "https://news.google.com/news/rss/headlines/section/topic/SCIENCE?ned=us&hl=en&gl=US"
health_news_url = "https://news.google.com/news/rss/headlines/section/topic/HEALTH?ned=us&hl=en&gl=US"
entertainment_news_url = "https://news.google.com/news/rss/headlines/section/topic/ENTERTAINMENT?ned=us&hl=en&gl=US"

# A dictionary with known topics mapped to a url from which we can harvest current headlines
topics_to_headlines_url_dict = {'Business': business_news_url, 
                                'Technology': tech_news_url,
                                'Science': science_news_url,
                                'Health': health_news_url,
                                'Entertainment': entertainment_news_url}

# A function to scrape current headlines and the google news category
def get_headlines(url):
    Client = urlopen(url)
    xml_page = Client.read()
    Client.close()
    
    soup_page = soup(xml_page, "html.parser")
    items = soup_page.findAll("item")
    title = soup_page.find("title")
    headlines = []
    categories = []
    for item in items:
        headlines.append(item.title.text)
        categories.append(title.text.split(' ')[0])
    
    df_headlines = pd.DataFrame({"Headline" : headlines, "Actual Category": categories})
    display("{} items in category {}".format(df_headlines.shape[0], categories[0]))   
    
    return pd.DataFrame({"Headline" : headlines, "Actual Category": categories})

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('tokenizer', tokenizer), ('classifier', classifier)])

In [ ]:
# Scrape current google news headlines
# of all known categories into a dataframe
# for prediction
df_inference = pd.DataFrame()
for key, value in topics_to_headlines_url_dict.items():
    df_inference = df_inference.append(get_headlines(value), ignore_index=True)

    
# Cut out all except 20 of the science and technology category rows
# to bring it more in line with other categories
number_to_remove = df_inference.loc[df_inference['Actual Category'].isin(['Science', 'Technology'])].shape[0] - 20
df_inference = df_inference[~df_inference['Headline'].isin(df_inference.loc[df_inference['Actual Category'].isin(['Science', 'Technology'])].sample(number_to_remove)['Headline'])].reset_index(drop=True)

# A function to convert category names to
# the model representations of the category names
def convert_category_name(x):
    if x == "Business":
        return 'b'
    elif x == "Science" or x == "Technology":
        return 't'
    elif x == "Health":
        return 'm'
    elif x == "Entertainment":
        return 'e'
    
df_inference['Actual Category'] = df_inference['Actual Category'].apply(lambda x: convert_category_name(x))

classifier
# pipeline.predict(df_inference.Headline.values)
# Add model predictions to the dataframe
# df_inference['Predicted Category'] = category_encoder.inverse_transform([x.argmax() for x in pipeline.predict(df_inference.Headline.values)])

In [ ]:
# Create files to store model configuration and weights
# such that the model can be built back up from the files
# rather than having to retrain.
# Thanks to https://machinelearningmastery.com/save-load-keras-deep-learning-models/
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
config_filename = "model_config_" + timestr + ".json"
weights_filename = "model_weights_" + timestr + ".h5"

# Serialize model to JSON
classifier_json = classifier.to_json()
with open(config_filename, "w") as json_file:
    json_file.write(classifier_json)

# Serialize weights to HDF5
classifier.save_weights(weights_filename)